# AHN Preprocessing
---
A notebook for preprocessing [AHN](https://www.ahn.nl) data.

#### Imports

In [ ]:
# Add project src to path.
import set_path

# Import modules.
from tqdm import tqdm
import pathlib
import numpy as np
import open3d as o3d

import preprocessing.ahn_preprocessing as ahn_preprocessing
from utils.interpolation import FastGridInterpolator
from utils import (
    ahn_utils,
    las_utils,
    math_utils,
    o3d_utils
)

#### Preprocess Folder
---

In [ ]:
# define AHN source folder
ahn_cloud_folder = '../../../../datasets/AHN/'

# define tree dataset folder
tree_dataset_folder = '../../dataset/'

In [ ]:
# collect ahn source files
ahn_data_folder = pathlib.Path(ahn_cloud_folder)
ahn_bboxes = las_utils.get_bbox_from_las_folder(ahn_data_folder)
print(f"- {len(ahn_bboxes)} AHN clouds: {list(ahn_bboxes.keys())}")

In [ ]:
# collect treecodes
data_folder = pathlib.Path(tree_dataset_folder)
surface_out_folder = data_folder.joinpath('ahn_surf/')
file_types = ('.LAS', '.las', '.LAZ', '.laz')
treecodes = set([las_utils.get_treecode_from_filename(f.name) for f in data_folder.glob('*/filtered_tree_*')
             if f.name.endswith(file_types)])
print(f"- {len(treecodes)} treecodes")


In [ ]:
# group trees per AHN file
treecodes_ahn = {key: [] for key in ahn_bboxes.keys()}
for treecode in treecodes:
    bbox = las_utils.get_bbox_from_tree_code(treecode, 10)
    for ahn_file, ahn_bbox in ahn_bboxes.items():
        if math_utils.bbox_in_bbox(bbox, ahn_bbox):
            treecodes_ahn[ahn_file].append(treecode)
            break

treecodes_ahn

In [ ]:
# Preprocess all treecodes
# WARNING: this may require 16+ GB of RAM, depending on the size of the clouda.
# WARNING: this may significant amount of time.
for ahn_file, treecodes in treecodes_ahn.items():
    print(ahn_file)
    if len(treecodes) > 0:
        ahn_cloud = las_utils.read_las(ahn_data_folder.joinpath(ahn_file))
        ahn_preprocessing.clip_ahn_treecodes(ahn_cloud, treecodes, surface_out_folder, buffer=10)